In [1]:
import numpy as np
import pandas as pd

In [2]:
state = pd.read_csv("./practical-statistics-for-data-scientists/data/state.csv")
state.head()

,State,Population,Murder.Rate,Abbreviation
0,Alabama,4779736,5.7,AL
1,Alaska,710231,5.6,AK
2,Arizona,6392017,4.7,AZ
3,Arkansas,2915918,5.6,AR
4,California,37253956,4.4,CA


## Measures of Location/Centrality
- mean
- median
- trimmed mean
- weighted mean

In [3]:
print("Arithmetic mean:", state.Population.mean())
print("The Median is:", state.Population.median())

Arithmetic mean: 6162876.3
The Median is: 4436369.5


In [4]:
from scipy.stats import hmean
# Locations measures of the murder rate
# Question: Are we sure we want to take the mean of a rate? Is there a better choice here?
# If memory serves, the appropriate way to take the average of a set of rates is the harmonic mean

# Measures of Location/Center
print("Arithmetic mean:", state["Murder.Rate"].mean())
print("The Median is:", state["Murder.Rate"].median())
print("Harmonic mean is:", hmean(state["Murder.Rate"]))

Arithmetic mean: 4.066
The Median is: 4.0
Harmonic mean is: 3.168854094682334


In [5]:
def weighted_mean(x, weights):
    x = pd.Series(x)
    weights = pd.Series(weights)
    numerator = np.dot(x, weights)
    denominator = weights.sum()
    return numerator / denominator

assert weighted_mean([1, 2, 3, 4, 5], [1, 1, 1, 1, 1]) == 3.0
assert weighted_mean([1, 2, 3, 4, 5], [5, 4, 3, 2, 1]) == 2.3333333333333335
assert weighted_mean([1, 2, 3], [0, 0, 1]) == 3
assert weighted_mean([1, 2, 3], [1, 0, 0]) == 1
assert weighted_mean([1, 2, 3], [1, 1, 0]) == 1.5

In [6]:
# Weighted mean with numpy
a = np.average(state["Murder.Rate"], weights=state["Population"])
print("Weighted mean is:", a)

Weighted mean is: 4.445833981123393


In [7]:
print("Weighted mean:", weighted_mean(state["Murder.Rate"], state.Population))

Weighted mean: 4.445833981123392


In [8]:
from scipy.stats import trim_mean
print("Population Mean:", state["Population"].mean())
print("5% Trimmed mean:", trim_mean(state["Population"], 0.05))
print("10% Trimmed mean:", trim_mean(state["Population"], 0.1))
print("20% Trimmed mean:", trim_mean(state["Population"], 0.2))

Population Mean: 6162876.3
5% Trimmed mean: 5316411.543478261
10% Trimmed mean: 4783697.125
20% Trimmed mean: 4413915.966666667


## Let's Measure Variability!
- "Mean absolute deviation" is what most folks think of when they think of deviations.
    - Ex. On any given day this week, the average change in high temperature is 5 degrees.
    - Ex. I average about 5 quarts of water a day, plus or minus one. One quart is the .mad() 
- Median absolute deviation from the median is another measure
- Variance
- Standard Deviation

In [9]:
x = pd.Series([1, 2, 3, 4, 5, 6, 7, 8, 9, 100])

print("Median is:", x.median())
print("Mean is:", x.mean())

Median is: 5.5
Mean is: 14.5


In [10]:
# Mean Absolute Deviation from the mean
# AKA Average absolute deviation from the mean
deviations = (x - x.mean()).abs()
deviations.mean()

assert deviations.mean() == x.mad()
print("Mean absolute deviation from the mean", x.mad())

Mean absolute deviation from the mean 17.1


In [11]:
# Average deviation from the median:
# This is less sensitibe to the outlier than the mean absolute deviation from the mean...
# This is still very sensitive to outliers, like the 1_000_000
deviations = (x - x.median()).abs()

print("Average deviation from the median is", deviations.mean())

Average deviation from the median is 11.5


In [12]:
# Median absolute deviation from the median
# SUPER robust to the high outlier
deviations = (x - x.median()).abs()
median_deviation_from_the_median = deviations.median()
print("Median deviation from the median is:", median_deviation_from_the_median)

Median deviation from the median is: 2.5


In [13]:
# Something about the relationships between these measures makes me want to dive deeper into Taleb's "Statistical Consequences of Fat Tails"

### The Gedek/Bruce/Bruce book says "Mean Absolute Deviation"  and the L1 Norm are Synonyms
- Mean absolute deviation from the mean is the same scalar as taking the l1 norm of that same vector?
- `x.mad() == np.linalg.norm(x, 1) / len(x)`

In [14]:
# The Gedek/Bruce/Bruce book says "Mean Absolute Deviation" is also known as the L1 norm or the Manhattan norm
# TODO: Prove it!
assert x.mad() == (x - x.mean()).abs().sum() / len(x)

print("Pandas .mad() is", x.mad())
print("Manual .mad() is", (x - x.mean()).abs().sum() / len(x))

Pandas .mad() is 17.1
Manual .mad() is 17.1


In [15]:
residuals = x - x.mean()
print("Numpy's L1 norm is", np.linalg.norm(residuals, 1)) 
print("Manual L1 norm is", (x - x.mean()).abs().sum()) # norm of the distance between x and x_bar

Numpy's L1 norm is 171.0
Manual L1 norm is 171.0


##### Takeaways
- Mean absolute deviation of a vector is not _exactly_ the same as the L1 norm of that vector
- Thee L1 norm is the numerator of mean absolute deviation. The denomminator is the length of the vector
- The only difference between L1 and .mad() is that .mad() divides the l1 by the number of observations

### Variance and Stardard Deviation
- based on squared deviations 
- Variance = $s^2$ = `np.sum(np.square(x - x.mean())) / n - 1` 
- Standard deviation = sqrt(variance) = $s$
- Standard deviation uses the same units as the original measure
- But most folk still say "standard deviation" when they have mean absolute deviation in their mind.

In [16]:
# The square root of the variance is the standard deviation
assert np.sqrt(x.var()) == x.std()
assert x.var() == np.sum(np.square(x - x.mean())) / (len(x) - 1)

### Now We Know
- variance > standard deviation > mean absolute deviation from the mean > median absolute deviation from the median

In [17]:
def median_absolute_deviation_from_median(x):
    x = pd.Series(x)
    
    # Obtain the residuals from the median (get the distance of the vector from the mean)
    residuals = (x - x.median()).abs()
    
    # obtain the median of the residuals
    return residuals.median()

assert median_absolute_deviation_from_median([1, 1, 1, 1]) == 0
assert median_absolute_deviation_from_median([1, 2, 3, 4, 5]) == 1 # because pd.Series([0, 1, 1, 2, 2]).median() is 1.0

In [18]:
# variance > standard deviation > mean absolute deviation > median absolute deviation from the median
assert x.var() > x.std() > x.mad() > median_absolute_deviation_from_median(x)

## Estimates Based on Percentiles
- _order statistics_ are measures based on sorted, ranked data. 
- Range, meaning `x.max() - x.min()` is the most basic measure
- The range is extemely sensitive to outliers and not an effective general measure of dispursion
- Range is less sensitive if we drop both the low and high outliers
- Quartiles
- IQR = range between Q3 and Q1 values


In [19]:
x.std()

30.152390728873666

In [20]:
q3 = x.quantile(.75)
q3

7.75

In [21]:
q1 = x.quantile(.25)
q1

3.25

In [22]:
iqr = q3 - q1
iqr

4.5

#### Other resources
- [David Lane's online stats resource has a section on percentiles](https://oreil.ly/o2fBI)
- [Kevin Davenport has a post about deviations from the median and robustness](https:/oreil.lyE7ZzcG)

### Vocabulary
- boxplot AKA box and whisker plot
- frequency table - tally count of numeric values falling into a set of intervals/bins
- histogram - frequency table of value counts
- density plot is a smoothed version of the histogram, AKA kernel density estimate or KDE